Import necessary libraries:

In [1]:
import sys
sys.path.append("..")
from data.preparer import load_news_dataset
from datetime import datetime
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import PandasLFApplier
from analyzer import train_model
import re
import pandas as pd
pd.set_option('display.max_colwidth', -1)
stat_history = pd.DataFrame()
import nltk
nltk.download("punkt")
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nofarcarmeli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nofarcarmeli/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
# Unzip the data. (Don't worry about this, it should be already unzipped.)
# Replace PASSWORD with the password to unzip the data, or download it directly from Kaggle.

#!unzip -P PASSWORD data/data.zip

In [3]:
# load the data
df_train, df_dev, df_valid, df_test = load_news_dataset()
print("{} training examples".format(len(df_train)))
print("{} development examples".format(len(df_dev)))
print("{} validation examples".format(len(df_valid)))
print("{} test examples".format(len(df_dev)))

477 training examples
400 development examples
100 validation examples
400 test examples


/Users/nofarcarmeli/Documents/GitHub/snorkel-notebooks/venv/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# Snorkel Tutorial

## News forum classification

### You will work with a subset of the 20 NewsGroup dataset. 
The texts shown are from one of two forums:
 1. Computer Electronics (Label 0)
 2. Gun Politics Forum (Label 1)
Your job is to create a training data set to classify texts as belonging to one of these two forums.

You will do this by writing labeling functions mapping text to 0 (ELECTRONICS), 1 (GUNS), or -1 (ABSTAIN or no label).
These functions will be aggregated by Snorkel to create training data from unlabeled examples.

You can evaluate your progress based on the coverage and f1 score of your label model, or by training a logistic regression classifier on the data and evaluating the test result.


In [4]:
# Start the timer!
stat_history.append({"time": datetime.now(), "num_lfs": 0}, ignore_index=True)

,num_lfs,time
0,0.0,2020-02-02 14:57:51.087355


Define the labels for this task:

In [5]:
ABSTAIN = -1
ELECTRONICS = 0
GUNS = 1

# Ready to Roll
Let's look at some positive and negative examples.

In [6]:
# rerun this cell to get a new sample
print("ELECTRONICS")
display(df_dev[df_dev.label==ELECTRONICS].sample(5))
print("GUNS")
display(df_dev[df_dev.label==GUNS].sample(5))

ELECTRONICS


,text,label
429,"Taft Electronics, 45th Street between 5th & 6th -- the only one left in\nwhat was once an entire district of electronics stores. A little expensive.\n\nTrans-Am Electronics, Canal Street near 7th Ave -- lots of surplus type\nstuff.\n\nSeveral other electronics or ""surplus"" type places are still on Canal\nStreet.\n\nI think Bronx Wholesale Radio is still in business -- Fordham Road not\ntoo far from Arthur Avenue in the Bronx. Also in the Bronx is NorthEastern\n(or was it Northwestern? Northeast Electronics?) on Jerome Avenue near\nBedford Park Boulevard. They're mostly a TV parts supply house, but when\nI was building CB radio projects, they were quite handy..",0
479,"\nDEC did this on their new alpha chip. I'm sure you could call them up\nand ask them how they did it (haha).\n\nActually, there are some details in their article in IEEE Journal of\nSolid-State Circuits. I think it was sometime around Nov. 1992.",0
453,"Hello,\n\nHas anyone used the Requirements Analsys methodology Hatley & Pirbhai ? I am\na British final year real-time degree student, and as a project I am covering\nthis methodology. I would be very grateful to anyone who could give me their\nviews on this method. Please mail me if you feel you can help, and I will send\nyou my questions.\n\nThanks in advance,",0
575,"\nRight. So all the cops will be buying antique muscle cars for chase cars;\notherwise the *police* cars will die too!\n\n-----------------------------------------------------------------------\n | To get the attention of a large animal, be it an\n | elephant or a bureaucracy, it helps to know what\nGeorge Patterson - | part of it feels pain. Be very sure, though, that\n | you want its full attention.\n | Kelvin Throop",0
415,"Just a few comments about the feasability of zipping up a bunch of miles\non your electronic odometer with an oscillator....\n\n I wouldn't expect to be able to do this. Not because the engineers at BMW are\nputting in anti-fraud logic, but just because an automobile is a noisy, hostile\nenvironment for electronics, and I would expect the software to be conservative.\n\n If I were to design a microprocessor-based odometer/speedometer program,\nit would have the following features ( just off the top of my head ):\n\n * Debouncing/deglitching ( Is this a real pulse, or just a glitch? )\n * Smoothing: The rate-of-change of speed cannot be too high:\n This is a car, not an electron.\n * Top speed limit: Anything above 200MPH is ridiculous: must be\n electrical noise or contact bounce\n\n As for getting in and presetting the pins: if this is the circuit board out of\na BMW dashboard, I seem to remember a single big chip with lotsa pins in the\nmiddle of the card. Good luck getting at the counter :-).\n\n I wouldn't expect a high input frequency to damage the meter: remember, it's\nprobably damped so it can't move too fast, and is probably protected. After all,\na car is an extremely hostile place for delicate electronics; there used to be an\napp note in the National Semiconductor Linear Databook that went into this\nin good detail.\n\n - Jerry Kaidor ( tr2!jerry@dragoman.com, jkaidor@synoptics.com )\n",0


GUNS


,text,label
548,"\n It's not the NRA's fault; but it is something to consider if you are\nconsidering contributing to the NRA. If candidate B is a complete asshole\nwhose only saving grace is that he opposes unnecessary restrictions on\nfirearms, I wouldn't want my membership dues funding efforts to get him\nre-elected.\n\n I have other problems with the NRA (as an organization; the individual\nmembers I've met have been loyal, trustworthy, honest, brave, etc.,\nespecially my boss who probably reads this newsgroup B->); they are\ndefinitely pro-hunting, and I recall seeing a pro-Desert Storm NRA bumper\nsticker. Sometimes they come on too strong in the political arena, which\ncontributes to their reputation as ""bad guys"" amoung many people.",1
504,"\n [flame-bait, pure and simple]\n\n\n",1
289,"\nI'd hardly call that ""giving up his chance to be Vice President of the US"";\nthe chance of the Populist Party ticket winning is essentially nil. Still,\nit does imply that he doesn't want to be associated with Duke.",1
283,": You are loosing.\n: There is no question about it. \n: Of those who vote, your cause is considered an abomination. No matter\n: how hard you try, public opinion is set against the RKBA. \n\n Care to show some *real* numbers instead of something HCI make up?\n I thought so, all ""foaming at the mouth"" shouting but nothing is \n ever said...\n \n: This is the end. By the finish of the Clinton administration, your\n: RKBA will be null and void. Tough titty.\n\n Well, we'll just have to wait and see about that, won't we? Or are\n you quite satisified with living in your little fantasy?\n \n< SNIP >\n: cases of firearms abuses has ruined your cause. There is nothing you\n< SNIP >\n: The press is against you, the public (the voting public) is against\n: you, the flow of history is against you ... this is it !\n\n Not true, it is ONLY those who are ignorant and are afraid to understand,\n accept, and deal with the real problems behind this violent society \n who are proposing gun control as a band-aid solution. May be I should \n refresh your memory with a quote from Prez. Clintion? \n\n ""It's the criminals, stupid!"" \n\n HEY, why is he cutting the budget for more prisons? May be someone need \n to remind him of what he promised...on second thought, why bother...\n\n: Surrender your arms. Soon enough, officers will be around to collect\n: them. Resistance is useless. They will overwhelm you - one at a time.\n: Your neighbors will not help you. They will consider you more if an\n: immediate threat than the abstract 'criminal'. \n\n Oooh, WACO II, coming to your living room soon... When was the last\n time you turned off your TV? Can't remember? I thought so...\n\n: Too fucking bad. You have gone the way of the KKK. Violent solutions\n: are passe'. Avoid situations which encourage criminals. Then you will\n: be as safe as possible. Such as it is ...\n\n Wait, I got it, this is a late April fool post, right? I didn't\n think ANYBODY is stupid enough to post something like this...good one\n guys, this group was getting boring without Holly and Susan.",1
327,"\nNot to mention last year's Weaver affair.\n\nAnyway, here's how I see the Waco affair; I'd be interested in other peoples'\ninterpretations...\n\n1. Koresh and his people were basically minding their own business.\n2. Some weapons violations may have been committed and I wouldn't have\n disapproved of prosecuting him for those violations. However, I think\n the BATF was criminal for starting negotiations with a military style\n assault and for firing into a house where there were children and other\n noncombatants.\n3. I don't see they couldn't just leave a token guard on the place and wait\n the BDs out; I don't approve of the tear gas approach and, if it caused\n the fire to be started, I think the FBI agent responsible should spend\n 10-20 years in jail.\n4. However, if Koresh's response to the tear gas was to kill everyone there,\n I hold him largely respon

## Writing Labeling Functions

__Your task for this tutorial is to write 5 labeling functions.__

Feel free to consult the internet or ask your experiment leader.

*(For the real task, you will be given 30 minutes. You will still be allowed to use the internet in this phase, but not ask your experiment leader.)*

Your function should take x as an input and output ELECTRONICS, GUNS, or ABSTAIN. 

In [7]:
@labeling_function()
def lf0(x):
    return GUNS if "firearm" in x.text.lower() else ABSTAIN

Your turn! try writing a function or editing the one above.

If you want, you can write helper functions to reuse. For example:

In [8]:
# example helper function 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
def sentiment(text):
    return sia.polarity_scores(text)["compound"]

In [9]:
@labeling_function()
def lf1(x):
    return ELECTRONICS if "tesla" in x.text.lower() else ABSTAIN

In [10]:
@labeling_function()
def lf2(x):
    count = len(re.findall("power", x.text.lower())) 
    return ELECTRONICS if count > 2 else GUNS 

In [11]:
@labeling_function()
def lf3(x):
    return ABSTAIN

In [12]:
@labeling_function()
def lf4(x):
    return ABSTAIN

Test your function (optional)

In [13]:
from types import SimpleNamespace

def test_func(lf, example_text):
    x = SimpleNamespace(text=example_text)
    return lf(x)

In [14]:
test_func(lf2, "I've got a powerful powerful tesla coil")

1

## Applying Functions
This is how we obtain training labels, by training a model to combine the outputs of the noisy labeling functions.
`L_train` and `L_dev` are matrices representing the label returned by each labeling function for each example in the training and development sets.

In [15]:
# Make sure this list has all your labeling functions
lfs = [lf0, lf1, lf2, lf3, lf4]

# Apply the LFs to the unlabeled training data, and the development data
applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)
L_dev = applier.apply(df_dev)

100%|██████████| 400/400 [00:00<00:00, 18476.70it/s]


Train the snorkel model to combine these noisy labels.

In [16]:
# Train the label model and compute the training labels
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
df_train["pred_label"] = label_model.predict_proba(L=L_train)[:,0]
probs_train = df_train["pred_label"]

# record intermediate results
# Don't worry about this code block, we just store some metrics to keep track of your progress.
Y_dev = df_dev.label.values
stats = label_model.score(L=L_dev, Y=Y_dev, metrics=["f1", "precision", "recall"])
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
            X=df_train, y=probs_train, L=L_train)
stats["training_label_coverage"] = len(probs_train_filtered)/len(probs_train)
stats["training_label_size"] = len(probs_train_filtered)
stats["time"] = datetime.now()
stat_history = stat_history.append(stats, ignore_index=True)

In [17]:
# let's see some examples of aggregated (probabilistic) labels!
# re run this cell for new examples
display(df_train.sample(5))

,text,pred_label
1008,"Yes, I know it sounds crazy. Call it an urge. Call it what you want.\nJust don't ask why :-)\n\nAnyway, I'd like to build a UV flashlight, cheaply. ""flashlight"" means\nit should be reasonably portable, but could have a power pack if\nnecessary.\n\nMy main question is the bulb: where can I get UV bulbs? Do they\nneed a lot of power? etc., etc.\n\nI'm not too concerned with whether it's long-wave or short-wave\n(but hey, if anyone has a cheap source of bulbs, I'll take both).\n\nOne other thing: a friend of mine mentioned something about near-UV\nlight being cheaper to get at than actual UV light. Does anyone\nknow what he was referring to?\n\nThanks much.\n\n",0.388506
1019,"\nAs a general rule, no relay will cleanly switch audio if you try to tranfer\nthe circuit with the contacts. The noise you hear is due to the momentary\nopening and closing of the path.\n\nThe noiseless way of transfering audio is to ground the circuit. In high\nimpedance audio circuits a resistive ""T"" is constructed close to characteristic\nimpedance of the circuit. Grounding the imputs (connected to the T) transfers\nthe audio.\n\nIn low impedance circuits transformers are usually used, and the inputs are\nshorted out or grounded. Secondaries are paralleled at the characteristic\nimpedance.\n\nSometimes if it is necessary to actually switch audio, a second contact is used\nto momentarily short the circuit output for the duration of the switching time.\n\nTelephone relays are handy, because contacts can be adjusted to ""Make before\nbreak and Vica Versa"" but I haven't seen any of these for years.\n\nNowadys switching is done electronically with OP amps, etc.\n\nA novel circuit I used to build was a primitive ""optical isolator"".. It consists\nof a resistive photocell and a lamp, all packaged in a tube. When the lamp is\noff the cell is high resistance. Turn the lamp on and the resistance lowers\npassing the audio. Once again this device in a ""T"" switches the audio. Varying\nthe lamp resistance give a remote volume control. Use 2 variable resisters and\nyou have a mixer!\n\nLots of luck!\n-- \n73, Tom\n================================================================================\nTom Wagner, Audio Visual Technician. Malaspina College Nanaimo British Columbia\n(604)753-3245, Loc 2230 Fax:755-8742 Callsign:VE7GDA Weapon:.45 Kentucky Rifle\nSnail mail to: Site Q4, C2. RR#4, Nanaimo, British Columbia, Canada, V9R 5X9",0.388506
618,"According to WNCI 97.9 FM radio this morning, Dayton, Ohio is operating a\ngun ""buy back"". They are giving $50 for every functional gun turned in.\nThey ran out of money in one day, and are now passing out $50 vouchers of\nsome sort. They are looking for more funds to keep operating. Another\nmedia-event brought to you by HCI.\n\nIs there something similar pro-gun people can do ? For example, pay $100\nto anyone who lawfully protects their life with a firearm ? Sounds a bit\ntacky, but hey, whatever works.",0.339520
1091,"\n\tIf alcohol were again banned today, it would be MUCH more\n\tdifficult to manage a large-scale smuggling operation. \n\tThe cops now rank just a narrow notch below the military\n\tin communications, intelligence gathering and firepower.\n\n\tIn a similar vein, the amount of marijuana smuggled into\n\tthis country has greatly decreased. This is because its\n\tvalue-per-pound is very low when compared to cocaine or\n\theroin. It's simply not worth the risk, it's uneconomical.\n\tNow, most reefer is domestic. There is less pressure on\n\tthe domestic producer (showy raids notwithstanding) and\n\tthus it is economical. Of note though ... domestic reefer\n\tis now very strong, so a small volume goes a long way.\n\tYou cannot make alcohol stronger than 200 proof - not a\n\tgood dollar/pound deal.",0.388506
669,"Here is a copy of my first update on the Randy Weaver trial.\nAfter a large response (about 15 email messages), I've decided\nthat there is sufficient interest here on t.p.g

## View Unlabeled Examples
You can use these to brainstorm new labeling functions. You may try filtering or sorting them in other ways.

If you get a `ValueError: a must be greater than 0 unless no samples are taken`, this means all your training examples are labeled by at least one LF.

In [18]:
# You can filter for unlabeled data
try:
    df_unlabeled = df_train[~df_train.index.isin(df_train_filtered.index)]
    display(df_unlabeled.sample(5))
except ValueError as e:
    print("ValueError: ")
    print(e)
    label_sums = (L_train != -1).sum(axis=1)
    print("\nExamples with lowest coverage: ({})".format(min(label_sums)))
    display(df_train[label_sums == min(label_sums)].head())

ValueError: 
a must be greater than 0 unless no samples are taken

Examples with lowest coverage: (1)


,text,pred_label
615,"I had a GE Emerson 13"" color TV for about 3 years and one fine day,\nwhile we were watching something (I doubt if the program was the cause),\nwe heard a mild explosion. Our screen went blank but there was sound,\nso we thought, 'oh we have special effects on the program'. But soon\nthe sound stopped and smoke started to appear at the back of the TV.\nThe brilliant EEs we are, we unplugged the TV and called customer service\nonly to be thrown around by please hold, I will transfer u to blah blah..\n Finally we abandoned the idea of trying to fix the TV and got a new one\n(we wanted a bigger one too!).\n After all the story, what I wanted to know is: Is my problem an isolated\nincident or a common one? (I recall reading about Russian TVs exploding, but\nnot here, in the US). Why would the picture tube explode or even smoke?\n I still have the left over TV set, I might dig into it this summer. Any\nidea where I can get parts for these things? (probably will cost more than TV).",0.388506
616,"\nAre you switching high level signals or low level signals like pre-amp\nout level signals? Also, are the clicks you mentioning the big\nclack that happens when it switches or are you refering to contact\nbounce? How are you driving the relays? TTL gate output? Switching\ntransistor? How are the relays connected to what you are driving?",0.388506
617,\n\n\n\nAnd they work especially well when the Feds have cut off your utilities.\n--,0.388506
619,"\nThey do. According the the Los Angeles Police Department, illegal\nmanufacture is one the three primary sources of machine guns and\nsubmachine guns used in crimes (sumggling and theft from the\npolice and military being the other two.) Washington D.C. police\nhave stated that 40% (If I'm remembering the figure correctly) of\nthe guns they conficate were illegally built.\n\n\nIt takes about 6 hours and a few tools to make one (at least one\nof reasonable quality). Unless the drug dealer enjoyes messing\naround on a lathe (say, as a hobby), he's going to have to \npay someone anyway. Materials plus six hours of a machinist's\ntime for something legal would run about $100. The blackmarket\nprices for guns are usually in the $50 to $200 range (at least\nthose few I've seen cited in newspaper articles were...)\n\n\nWashington D.C. has a total ban on handguns and prohibits assembled\nrifles within city limits. It's homicide rate is almost ten\ntimes the national average. It is also illegal for a D.C.\nresident to drive to the Virginia or Maryland suburbs and\nbuy a gun (dealers are required by federal law to check IDs\nand make sure the buyer isn't from another state) so the ban \ncan not be easily side-stepped.\n\n\nSure. But it you couldn't buy one, you would write your own (in\nfact, people _did_ write their own 15 years ago...) More likely,\nyou would find a friend who was a particularly good programer\nand get him give you a copy of his. Software is a _very_ bad\nexample for your case: How many people do you know with illegal\ncopies of $400 word processors? If people want something, and\nit isn't available (or affordable) legally, they will usually\nget it illegally.\n\n\nQuite.\n\n\n13-3-2. The formula has been around for half a million years.\nOr are you going to restrict sales of sulpher, charcoal and\nsaltpeter? That's alot cruder than modern smokless powder, but\nit works very well. The only real problems are a ~25% reduction\nin energy (so a .357 magnum would ""only"" be as deadly as a 9mm)\nand it makes alot more smoke... Of course, a smart black marketeer\ncould just make the gun in 9mm and steal the ammunition from the\npolice (the police are often corruptable, and things are known\nto disappear from police evidence rooms and armories and\nreappear on the streets...)\n\n\nPerhaps you weren't watching the news two years ago, but the Serbs \nalso tried to invade Slovinia. They were driven out after a few\nweeks by partisans armed with home-made _anti-tank_ weapons.

## Analyze Results
Evaluate the accuracy of the estimated training labels and development set labels (based on ground truth).

`Polarity` describes the set of outputs of each function, not including `ABSTAIN (-1)`.
For example, a function that returns `ABSTAIN` or `GUNS` has polarity `[1]`

In [19]:
train_analysis = LFAnalysis(L=L_train, lfs=lfs).lf_summary()
display("Training set results:", train_analysis)

'Training set results:'

,j,Polarity,Coverage,Overlaps,Conflicts
lf0,0,[1],0.088050,0.088050,0.000000
lf1,1,[0],0.002096,0.002096,0.002096
lf2,2,"[0, 1]",1.000000,0.090147,0.002096
lf3,3,[],0.000000,0.000000,0.000000
lf4,4,[],0.000000,0.000000,0.000000


In [20]:
Y_dev = df_dev.label.values
dev_analysis = LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)
display("Dev set results:", dev_analysis)

'Dev set results:'

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
lf0,0,[1],0.0675,0.0675,0.000,27,0,1.00
lf1,1,[0],0.0050,0.0050,0.005,2,0,1.00
lf2,2,"[0, 1]",1.0000,0.0725,0.005,190,0,0.48
lf3,3,[],0.0000,0.0000,0.000,0,0,0.00
lf4,4,[],0.0000,0.0000,0.000,0,0,0.00


## Save the Model
When you have finished

In [21]:
label_model.save("snorkel_tutorial_lfmodel.pkl")
stat_history.to_csv("snorkel_tutorial_statistics_history.csv")

## Train Model
We can train a simple bag of words model on these labels, and see test accuracy.

(This step may take a while).

In [22]:
train_model(label_model, df_train, df_valid, df_test, L_train)

477 out of 477 examples used for training data


Restoring model weights from the end of the best epoch.
Epoch 00032: early stopping


Test Accuracy: 60.0%


In [23]:
print(L_train)

[[ 1 -1  1 -1 -1]
 [ 1 -1  1 -1 -1]
 [-1 -1  1 -1 -1]
 ...
 [-1 -1  1 -1 -1]
 [-1 -1  1 -1 -1]
 [-1 -1  1 -1 -1]]
